# Saumya Kothari - Natural Language Processing (NLP) Project 1 (Part 2)

#### DOMAIN: 
Customer support

#### CONTEXT: 
Great Learning has a an academic support department which receives numerous support requests every day throughout the
year. Teams are spread across geographies and try to provide support round the year. Sometimes there are circumstances where due to
heavy workload certain request resolutions are delayed, impacting company’s business. Some of the requests are very generic where a
proper resolution procedure delivered to the user can solve the problem. Company is looking forward to design an automation which can
interact with the user, understand the problem and display the resolution procedure [ if found as a generic request ] or redirect the request
to an actual human support executive if the request is complex or not in it’s database.

#### DATA DESCRIPTION: 
A sample corpus is attached for your reference. Please enhance/add more data to the corpus using your linguistics
skills.

#### PROJECT OBJECTIVE: 
Design a python based interactive semi - rule based chatbot which can do the following:
1. Start chat session with greetings and ask what the user is looking for.
2. Accept dynamic text based questions from the user. Reply back with relevant answer from the designed corpus.
3. End the chat session only if the user requests to end else ask what the user is looking for. Loop continues till the user asks to end it.
Please use the sample chatbot demo video for reference.

#### EVALUATION: 
GL evaluator will use linguistics to twist and turn sentences to ask questions on the topics described in DATA DESCRIPTION
and check if the bot is giving relevant replies.

In [1]:
# IMPORT NECESSARY DATA PACKAGES
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [2]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('GL Bot.json').read()
intents = json.loads(data_file)

In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [4]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")

# classes = intents
print (len(classes), "classes", classes)

# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

128 documents
8 classes ['Bot', 'Exit', 'Intro', 'NN', 'Olympus', 'Profane', 'SL', 'Ticket']
158 unique lemmatized words ['a', 'able', 'access', 'activation', 'ada', 'adam', 'aifl', 'aiml', 'am', 'an', 'ann', 'anyone', 'are', 'artificial', 'backward', 'bad', 'bagging', 'batch', 'bayes', 'belong', 'best', 'blended', 'bloody', 'boosting', 'bot', 'buddy', 'classification', 'contact', 'create', 'cross', 'cya', 'day', 'deep', 'did', 'diffult', 'do', 'ensemble', 'epoch', 'explain', 'first', 'for', 'forest', 'forward', 'from', 'function', 'good', 'goodbye', 'gradient', 'great', 'hate', 'have', 'hell', 'hello', 'help', 'helped', 'hey', 'hi', 'hidden', 'hour', 'how', 'hyper', 'i', 'imputer', 'in', 'intelligence', 'is', 'jerk', 'joke', 'knn', 'later', 'layer', 'learner', 'learning', 'leaving', 'link', 'listen', 'logistic', 'lot', 'machine', 'me', 'ml', 'my', 'naive', 'name', 'nb', 'net', 'network', 'neural', 'no', 'not', 'of', 'olympus', 'olypus', 'on', 'online', 'operation', 'opertions', 'otimi

In [6]:
# create our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
    
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-6-c136eb46542c>:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [7]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
26/26 [==============================] - 2s 2ms/step - loss: 2.1206 - accuracy: 0.1369
Epoch 2/200
26/26 [==============================] - 0s 2ms/step - loss: 1.9477 - accuracy: 0.1766
Epoch 3/200
26/26 [==============================] - 0s 3ms/step - loss: 1.8527 - accuracy: 0.2392
Epoch 4/200
26/26 [==============================] - 0s 2ms/step - loss: 1.7678 - accuracy: 0.3011
Epoch 5/200
26/26 [==============================] - 0s 3ms/step - loss: 1.7930 - accuracy: 0.2795
Epoch 6/200
26/26 [==============================] - 0s 3ms/step - loss: 1.7791 - accuracy: 0.3204
Epoch 7/200
26/26 [==============================] - 0s 2ms/step - loss: 1.5971 - accuracy: 0.4138
Epoch 8/200
26/26 [==============================] - 0s 3ms/step - loss: 1.4457 - accuracy: 0.4418
Epoch 9/200
26/26 [==============================] - 0s 3ms/step - loss: 1.3123 - accuracy: 0.5083
Epoch 10/200
26/26 [==============================] - 0s 3ms/step - loss: 1.3054 - accuracy: 0.5936
Epoch 11/

26/26 [==============================] - 0s 1ms/step - loss: 0.0383 - accuracy: 0.9929
Epoch 83/200
26/26 [==============================] - 0s 932us/step - loss: 0.0389 - accuracy: 0.9847
Epoch 84/200
26/26 [==============================] - 0s 3ms/step - loss: 0.0307 - accuracy: 0.9860
Epoch 85/200
26/26 [==============================] - 0s 3ms/step - loss: 0.0698 - accuracy: 0.9921
Epoch 86/200
26/26 [==============================] - 0s 3ms/step - loss: 0.0259 - accuracy: 1.0000
Epoch 87/200
26/26 [==============================] - 0s 2ms/step - loss: 0.0156 - accuracy: 0.9941
Epoch 88/200
26/26 [==============================] - 0s 2ms/step - loss: 0.0416 - accuracy: 0.9823
Epoch 89/200
26/26 [==============================] - 0s 2ms/step - loss: 0.0145 - accuracy: 1.0000
Epoch 90/200
26/26 [==============================] - 0s 1ms/step - loss: 0.0241 - accuracy: 0.9970
Epoch 91/200
26/26 [==============================] - 0s 2ms/step - loss: 0.0094 - accuracy: 1.0000
Epoch 92/20

26/26 [==============================] - 0s 3ms/step - loss: 0.0051 - accuracy: 1.0000
Epoch 164/200
26/26 [==============================] - 0s 2ms/step - loss: 0.0049 - accuracy: 1.0000
Epoch 165/200
26/26 [==============================] - 0s 3ms/step - loss: 0.0116 - accuracy: 1.0000
Epoch 166/200
26/26 [==============================] - 0s 2ms/step - loss: 0.0076 - accuracy: 1.0000
Epoch 167/200
26/26 [==============================] - 0s 2ms/step - loss: 0.0113 - accuracy: 1.0000
Epoch 168/200
26/26 [==============================] - 0s 2ms/step - loss: 0.0123 - accuracy: 1.0000
Epoch 169/200
26/26 [==============================] - 0s 2ms/step - loss: 0.0130 - accuracy: 1.0000
Epoch 170/200
26/26 [==============================] - 0s 2ms/step - loss: 0.0183 - accuracy: 0.9844
Epoch 171/200
26/26 [==============================] - 0s 1ms/step - loss: 0.0057 - accuracy: 1.0000
Epoch 172/200
26/26 [==============================] - 0s 1ms/step - loss: 0.0085 - accuracy: 1.0000
Epoc

In [8]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('GL Bot.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [9]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [10]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [11]:
def chat():
    print("Chat with Bot(type: stop to quit)")
    print("If answer is not right (type: *)")
    print("\nWelcome to the Chat! How can I help you?")
    while True:
        inp = input("\n\nYou: ")
        if inp.lower()=="*":
            print("Bot: Please rephrase your question and try again")
        elif inp.lower() == "quit":
            break
        else:
            results = chatbot_response(inp)
            print("Bot: "+ results)

chat()

Chat with Bot(type: stop to quit)
If answer is not right (type: *)

Welcome to the Chat! How can I help you?


You: Please help me
Bot: Hello! how can i help you ?


You: I am not able to understand olympus
Bot: Link: Olympus wiki


You: explain me how machine learning works
Bot: Link: Machine Learning wiki 


You: logistic regression
Bot: Link: Machine Learning wiki 


You: who are you
Bot: I am your virtual learning assistant


You: i am not able to understand deep learning
Bot: Link: Neural Nets wiki


You: *
Bot: Please rephrase your question and try again


You: epoch
Bot: Link: Neural Nets wiki


You: not a good solution
Bot: Tarnsferring the request to your PM


You: thank you
Bot: I hope I was able to assist you, Good Bye


You: Quit
